In [22]:
import pandas as pd
import json
import matplotlib.pyplot as plt
import seaborn as sns

from lazypredict.Supervised import LazyClassifier 
from sklearn.preprocessing import LabelEncoder
from datetime import datetime

from  sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression

import warnings
warnings.filterwarnings("ignore")



In [23]:
with open("D:\DS\Projects\json_data\databangladeshi_users.json") as f :
    jsn = json.load(f)

jsn

[{'user_id': 1001,
  'purchases': [1994, 651, 1246, 1040],
  'location': 'Barisal',
  'last_login': '2025-04-25',
  'loyalty': 'Bronze'},
 {'user_id': 1002,
  'purchases': [645, 492],
  'location': 'Chittagong',
  'last_login': '2025-04-30',
  'loyalty': 'Bronze'},
 {'user_id': 1003,
  'purchases': [1285, 1198],
  'location': 'Rajshahi',
  'last_login': '2025-05-06',
  'loyalty': 'Bronze'},
 {'user_id': 1004,
  'purchases': [114, 1662, 210],
  'location': 'Mymensingh',
  'last_login': '2025-03-24',
  'loyalty': 'Bronze'},
 {'user_id': 1005,
  'purchases': [1269, 1418, 559, 835, 781],
  'location': 'Khulna',
  'last_login': '2025-05-20',
  'loyalty': 'Bronze'},
 {'user_id': 1006,
  'purchases': [1341, 1478],
  'location': 'Mymensingh',
  'last_login': '2025-01-25',
  'loyalty': 'Gold'},
 {'user_id': 1007,
  'purchases': [685, 1836, 245, 632],
  'location': 'Rajshahi',
  'last_login': '2025-04-23',
  'loyalty': 'Silver'},
 {'user_id': 1008,
  'purchases': [1746],
  'location': 'Dhaka',
 

In [24]:
flat_data = []

for user_data in jsn :
    flat_data.append({
        "purchases" : sum(user_data["purchases"]),
        "items" : len(user_data["purchases"]),
        "average" : sum(user_data["purchases"])/len(user_data["purchases"]),
        "location" : user_data["location"],
        "lastlogin" : user_data["last_login"],
        "loyalty" : user_data["loyalty"]
    })

df = pd.DataFrame(flat_data)
df.head()

,purchases,items,average,location,lastlogin,loyalty
0,4931,4,1232.75,Barisal,2025-04-25,Bronze
1,1137,2,568.50,Chittagong,2025-04-30,Bronze
2,2483,2,1241.50,Rajshahi,2025-05-06,Bronze
3,1986,3,662.00,Mymensingh,2025-03-24,Bronze
4,4862,5,972.40,Khulna,2025-05-20,Bronze


Cleaning and encoding

In [25]:
df["lastlogin"] = pd.to_datetime(df["lastlogin"])

df["lastseen"] = (datetime.today() - df["lastlogin"]).dt.days


df = df.drop(columns= "lastlogin")

df.head()

,purchases,items,average,location,loyalty,lastseen
0,4931,4,1232.75,Barisal,Bronze,30
1,1137,2,568.50,Chittagong,Bronze,25
2,2483,2,1241.50,Rajshahi,Bronze,19
3,1986,3,662.00,Mymensingh,Bronze,62
4,4862,5,972.40,Khulna,Bronze,5


In [26]:
le_loc = LabelEncoder()
le_loyl = LabelEncoder()

df["location"] = le_loc.fit_transform(df["location"])

df["loyalty"] = le_loyl.fit_transform(df["loyalty"])

df.head()

,purchases,items,average,location,loyalty,lastseen
0,4931,4,1232.75,0,0,30
1,1137,2,568.50,1,0,25
2,2483,2,1241.50,5,0,19
3,1986,3,662.00,4,0,62
4,4862,5,972.40,3,0,5


In [27]:
features = ['purchases', 'items', 'average', 'location', 'lastseen']
X_scaled = StandardScaler().fit_transform(df[features])

splitting

In [28]:

x = df.drop("loyalty",axis=1)
y = df["loyalty"]

x_train , x_test , y_train ,y_test = train_test_split(x,y,test_size=.2)

lazypredict

In [29]:

model = LazyClassifier(ignore_warnings=True)

models, predict = model.fit(x_train, x_test, y_train, y_test)
models

  0%|          | 0/31 [00:00<?, ?it/s]

[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000038 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 375
[LightGBM] [Info] Number of data points in the train set: 400, number of used features: 5
[LightGBM] [Info] Start training from score -1.108663
[LightGBM] [Info] Start training from score -1.101115
[LightGBM] [Info] Start training from score -1.086190
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No furt

,Accuracy,Balanced Accuracy,ROC AUC,F1 Score,Time Taken
Model,,,,,
PassiveAggressiveClassifier,0.41,0.40,None,0.39,0.01
Perceptron,0.38,0.38,None,0.37,0.01
SVC,0.38,0.38,None,0.38,0.02
CalibratedClassifierCV,0.39,0.38,None,0.35,0.04
LGBMClassifier,0.36,0.36,None,0.35,0.07
NuSVC,0.36,0.36,None,0.35,0.05
KNeighborsClassifier,0.35,0.36,None,0.33,0.02
RidgeClassifierCV,0.36,0.36,None,0.30,0.02
LabelPropagation,0.35,0.36,None,0.34,0.01
